In [1]:
import os
import json
import uuid
import asyncio
from datetime import datetime
import boto3
from boto3.dynamodb.conditions import Key, Attr
from botocore.exceptions import ClientError
from typing import Dict, List, Optional, Any
import logging


In [2]:
# os.environ["AWS_REGION"] = "eu-west-2" 
# os.environ["DYNAMODB_TABLE_NAME"] = "feedback"

In [3]:
region = "us-west-2"
table_name = "feedback"

dynamodb = boto3.resource('dynamodb', region_name=region)
table = dynamodb.Table(table_name)

---

In [4]:
dynamodb = boto3.resource('dynamodb',region_name=region)
dynamodb

dynamodb.ServiceResource()

In [5]:
table = dynamodb.Table(table_name)
table

dynamodb.Table(name='feedback')

In [6]:
feedback_id = str(uuid.uuid4())
feedback_id

'ed56dd22-6784-49cd-9c23-d3b3b1fefaa5'

In [7]:
item = {
    'feedback_id':feedback_id,
    'text': "Feedback 01"
}
response = table.put_item(Item=item)
response

{'ResponseMetadata': {'RequestId': 'S58B4NVI6QHO9N6FF2A0QBGQNBVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Mon, 03 Mar 2025 19:07:40 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'S58B4NVI6QHO9N6FF2A0QBGQNBVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2745614147'},
  'RetryAttempts': 0}}

In [8]:
response = table.get_item(Key={'feedback_id': feedback_id})
response

{'Item': {'feedback_id': 'ed56dd22-6784-49cd-9c23-d3b3b1fefaa5',
  'text': 'Feedback 01'},
 'ResponseMetadata': {'RequestId': 'N7BGH6J8NQPNGB8PEV6STK1T8BVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Mon, 03 Mar 2025 19:07:40 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '96',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'N7BGH6J8NQPNGB8PEV6STK1T8BVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '1131761939'},
  'RetryAttempts': 0}}

In [9]:
timestamp = datetime.now().isoformat()
new_text = "Updated Text"

response = table.update_item(
    Key={"feedback_id":feedback_id},
    UpdateExpression= "SET #txt = :text, updated_at = :timestamp",
    ExpressionAttributeNames={
        '#txt':'text'
    },
    ExpressionAttributeValues={
        ':text': new_text,
        ':timestamp': timestamp
    },
    ReturnValues="UPDATED_NEW"
)
response

{'Attributes': {'updated_at': '2025-03-03T19:07:41.603584',
  'text': 'Updated Text'},
 'ResponseMetadata': {'RequestId': 'V834BF67HLTSUSPD249T24M353VV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Mon, 03 Mar 2025 19:07:40 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '92',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'V834BF67HLTSUSPD249T24M353VV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '127592059'},
  'RetryAttempts': 0}}

In [10]:
Limit = 100

response = table.scan(Limit=Limit)
response.get('Items','no data')

[{'updated_at': '2025-03-03T19:07:41.603584',
  'feedback_id': 'ed56dd22-6784-49cd-9c23-d3b3b1fefaa5',
  'text': 'Updated Text'}]

Delete

In [ ]:
response = table.delete_item(
    Key={'feedback_id': feedback_id},
    ReturnValues="ALL_OLD"
)

response

In [15]:
import boto3

dynamodb = boto3.resource('dynamodb', region_name="us-west-2")
table = dynamodb.Table("feedback")

response = table.scan()
items = response.get('Items', [])

while 'LastEvaluatedKey' in response:
    response = table.scan(ExclusiveStartKey=response['LastEvaluatedKey'])
    items.extend(response.get('Items', []))

with table.batch_writer() as batch:
    for item in items:
        batch.delete_item(Key={'feedback_id': item['feedback_id']})

print(f"Deleted {len(items)} items")

Deleted 100 items


---
---

https://docs.aws.amazon.com/amazondynamodb/latest/developerguide/best-practices.html

In [16]:
import boto3, uuid, random, json
from datetime import datetime, timedelta

dynamodb = boto3.resource('dynamodb', region_name="us-west-2").Table("feedback")

for i in range(100):
    created_at = (datetime.now() - timedelta(days=random.randint(1, 365))).isoformat()
    feedback_type = random.choice([None, "guardrail_failure", "performance_issue", "usability"])
    
    dynamodb.put_item(Item={
        'feedback_id': str(uuid.uuid4()),
        'text': json.dumps({
            "RACFID": f"USER00{random.randint(1, 5)}",
            "InferenceID": f"inf-{uuid.uuid4().hex[:12]}",
            "ApplicationID": random.choice(["APP1", "APP2", "APP3", None]),
            "feedbackRelevance": random.choice(["0", "1", None]),
            "feedbackType": feedback_type,
            "guardrailFailureType": random.choice(["profanity", "bias", "hallucination", None]) if feedback_type == "guardrail_failure" else None,
            "description": random.choice(["Model hallucinated data", "System was slow", None]),
            "context": {"model_version": f"v{random.randint(1,5)}.{random.randint(0,9)}", "latency_ms": random.randint(50, 2000)}
        }),
        'created_at': created_at
    })


In [8]:
from boto3.dynamodb.conditions import Attr
import boto3

table = boto3.resource('dynamodb', region_name="us-west-2").Table("feedback")
filter_expression = Attr('RACFID').contains('USER')

response, item_list = {}, []
while True:
    scan_kwargs = {"FilterExpression": filter_expression}
    if 'LastEvaluatedKey' in response:
        scan_kwargs["ExclusiveStartKey"] = response['LastEvaluatedKey']

    response = table.scan(**scan_kwargs)
    item_list.extend(response.get('Items', []))

    if 'LastEvaluatedKey' not in response:
        break

print(f"Total items fetched: {len(item_list)}")


Total items fetched: 0


In [13]:
response, total_count ={}, 0

while True:
    scan_kwargs = {"Select":"COUNT"}
    if 'LastEvaluatedKey' in response:
        scan_kwargs["ExclusiveStartKey"] = response['LastEvaluatedKey']
    
    response = table.scan(**scan_kwargs)

    total_count +=response.get('Count',0)

    if 'LastEvaluatedKey' not in response:
        break

print(response,'\n',total_count)

{'Count': 100, 'ScannedCount': 100, 'ResponseMetadata': {'RequestId': 'ERA0RSFA7CVJP8QE8K2N194LUNVV4KQNSO5AEMVJF66Q9ASUAAJG', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'Server', 'date': 'Tue, 04 Mar 2025 07:19:40 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '32', 'connection': 'keep-alive', 'x-amzn-requestid': 'ERA0RSFA7CVJP8QE8K2N194LUNVV4KQNSO5AEMVJF66Q9ASUAAJG', 'x-amz-crc32': '1321842452'}, 'RetryAttempts': 0}} 
 100


get all records/ scan() with pagination

In [ ]:
import boto3

table = boto3.resource('dynamodb', region_name="us-west-2").Table("feedback")

response, all_items = {}, []
while True:
    scan_kwargs = {}
    if 'LastEvaluatedKey' in response:
        scan_kwargs["ExclusiveStartKey"] = response['LastEvaluatedKey']

    response = table.scan(**scan_kwargs)
    all_items.extend(response.get('Items', []))

    if 'LastEvaluatedKey' not in response:
        break

print(len(all_items))

100
{'Items': [{'feedback_id': 'dd8cae11-2a29-4969-982e-8a196b2751ba', 'text': '{"RACFID": "USER005", "InferenceID": "inf-b4b4c88001ea", "ApplicationID": "APP3", "feedbackRelevance": "1", "feedbackType": null, "guardrailFailureType": null, "description": "System was slow", "context": {"model_version": "v2.1", "latency_ms": 1762}}', 'created_at': '2024-11-16T19:31:24.492223'}, {'feedback_id': '9048640b-cef0-48b4-9b25-41c4bb42dc75', 'text': '{"RACFID": "USER001", "InferenceID": "inf-721153a049b5", "ApplicationID": "APP3", "feedbackRelevance": "0", "feedbackType": null, "guardrailFailureType": null, "description": "Model hallucinated data", "context": {"model_version": "v1.1", "latency_ms": 1530}}', 'created_at': '2024-08-03T19:31:31.163465'}, {'feedback_id': '7f75ba0e-8825-4a67-998a-4a4904bcaa0e', 'text': '{"RACFID": "USER001", "InferenceID": "inf-d5b81a1ea670", "ApplicationID": "APP3", "feedbackRelevance": "1", "feedbackType": "guardrail_failure", "guardrailFailureType": "hallucination"

In [18]:
all_items

[{'feedback_id': 'dd8cae11-2a29-4969-982e-8a196b2751ba',
  'text': '{"RACFID": "USER005", "InferenceID": "inf-b4b4c88001ea", "ApplicationID": "APP3", "feedbackRelevance": "1", "feedbackType": null, "guardrailFailureType": null, "description": "System was slow", "context": {"model_version": "v2.1", "latency_ms": 1762}}',
  'created_at': '2024-11-16T19:31:24.492223'},
 {'feedback_id': '9048640b-cef0-48b4-9b25-41c4bb42dc75',
  'text': '{"RACFID": "USER001", "InferenceID": "inf-721153a049b5", "ApplicationID": "APP3", "feedbackRelevance": "0", "feedbackType": null, "guardrailFailureType": null, "description": "Model hallucinated data", "context": {"model_version": "v1.1", "latency_ms": 1530}}',
  'created_at': '2024-08-03T19:31:31.163465'},
 {'feedback_id': '7f75ba0e-8825-4a67-998a-4a4904bcaa0e',
  'text': '{"RACFID": "USER001", "InferenceID": "inf-d5b81a1ea670", "ApplicationID": "APP3", "feedbackRelevance": "1", "feedbackType": "guardrail_failure", "guardrailFailureType": "hallucination", 

In [9]:
boto3.resource('dynamodb', region_name="us-west-2").get_available_subresources()

['Table']

In [5]:
import requests
import json
import uuid
import random
from datetime import datetime

APPLICATION_IDS = ["chatbot-v1", "customer-support-ai", "research-assistant"]
RACFIDS = ["user-001", "user-002", "admin-001"]
FEEDBACK_TYPES = ["performance_issue", "usability", "guardrail_failure"]
GUARDRAIL_FAILURE_TYPES = ["profanity", "bias", "inaccuracy/hallucination"]

def generate_feedback_payload():
    return {
        "inferenceId": str(uuid.uuid4()),
        "racfId": random.choice(RACFIDS),
        "applicationId": random.choice(APPLICATION_IDS),
        "timestamp": int(datetime.now().timestamp()),
        "feedbackRelevance": random.choice(["0", "1"]),
        "feedbackType": random.choice(FEEDBACK_TYPES),
        "guardrailFailureType": random.choice(GUARDRAIL_FAILURE_TYPES) if random.random() < 0.3 else None,
        "description": f"Feedback for {random.choice(APPLICATION_IDS)}",
        "context": {
            "timestamp": datetime.now().isoformat() + "Z",
            "userAgent": "TestScript/1.0",
            "ipAddress": f"192.168.1.{random.randint(1, 255)}",
            "additionalMetadata": {
                "session": str(uuid.uuid4()),
                "interactionCount": random.randint(1, 50)
            }
        }
    }

def submit_feedback(url, payload):
    try:
        response = requests.post(url, 
                                 headers={'Content-Type': 'application/json'},
                                 data=json.dumps(payload))
        response.raise_for_status()
        return response.json()
    except requests.RequestException as e:
        print(f"Error: {e}")
        return None

def main():
    BASE_URL = "http://127.0.0.1:8000/v1/feedback/"
    for i in range(20):
        payload = generate_feedback_payload()
        result = submit_feedback(BASE_URL, payload)
        print(f"Submission {i+1}: {result}")

if __name__ == "__main__":
    main()

Error: 422 Client Error: Unprocessable Entity for url: http://127.0.0.1:8000/v1/feedback/
Submission 1: None
Error: 422 Client Error: Unprocessable Entity for url: http://127.0.0.1:8000/v1/feedback/
Submission 2: None
Error: 422 Client Error: Unprocessable Entity for url: http://127.0.0.1:8000/v1/feedback/
Submission 3: None
Error: 422 Client Error: Unprocessable Entity for url: http://127.0.0.1:8000/v1/feedback/
Submission 4: None
Error: 422 Client Error: Unprocessable Entity for url: http://127.0.0.1:8000/v1/feedback/
Submission 5: None
Error: 422 Client Error: Unprocessable Entity for url: http://127.0.0.1:8000/v1/feedback/
Submission 6: None
Error: 422 Client Error: Unprocessable Entity for url: http://127.0.0.1:8000/v1/feedback/
Submission 7: None
Error: 422 Client Error: Unprocessable Entity for url: http://127.0.0.1:8000/v1/feedback/
Submission 8: None
Error: 422 Client Error: Unprocessable Entity for url: http://127.0.0.1:8000/v1/feedback/
Submission 9: None
Error: 422 Client E

In [ ]:

# Curl equivalent for single submission
"""
curl -X 'POST' \
  'http://127.0.0.1:8000/v1/feedback/' \
  -H 'accept: application/json' \
  -H 'Content-Type: application/json' \
  -d '{
  "RACFID": "user-001",
  "InferenceID": "inference-123-456",
  "ApplicationID": "chatbot-v1",
  "feedbackRelevance": "1",
  "feedbackType": "performance_issue",
  "guardrailFailureType": "hallucination",
  "description": "AI provided inaccurate information about recent events",
  "context": {
    "timestamp": "2025-03-04T12:00:00Z",
    "userAgent": "Chrome/91.0",
    "ipAddress": "192.168.1.100",
    "additionalMetadata": {
      "session": "session-789-012",
      "interactionCount": 15
    }
  }
}'
"""

# OpenAPI/Swagger Test Payload Example
"""

"""

In [ ]:
!openapi-python-client generate --path openapi.yaml

In [ ]:
from op